In [ ]:
import nltk
import pandas as pd

# Download corpus
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)

dt = pd.read_csv('./STS.input.SMTeuroparl.txt',sep='\t',header=None)
gold_standard_dt = pd.read_csv('./STS.gs.SMTeuroparl.txt',sep='\t',header=None)

wnl = nltk.stem.WordNetLemmatizer()
def lemmatize(p):
  d = {'NN': 'n', 'NNS': 'n', 
       'JJ': 'a', 'JJR': 'a', 'JJS': 'a', 
       'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v', 
       'RB': 'r', 'RBR': 'r', 'RBS': 'r'}
  
  # decide what to do with the rest (this version ignores them)
  if p[1] in d:
    return wnl.lemmatize(p[0], pos=d[p[1]])
  return p[0]

# when comparing should do the same preprocessing. If we do preprocessing before posttagger its wrong cause its trained on normal sentences. Order is important. If we want to apply the same preprocessing as previous session first tokenize and get post-tags and then we can apply anything we want.
# list of post tags is different per model 
# will lemmas always be better? if we think no give counterexample

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/pandelissymeonidis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pandelissymeonidis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/pandelissymeonidis/nltk_data...
